In [35]:
%matplotlib inline

import numpy as np
import pandas as pd
import spacy #nlp module
import re
import matplotlib as plot #visuals

import sklearn as skl
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier # Decision Trees

from collections import defaultdict, Counter

import nltk as nltk #nlp module
from nltk import sent_tokenize, word_tokenize, pos_tag
from nltk.corpus import treebank
from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

import random
from random import sample

nlp = spacy.load('en')
from spacy.en import English
parser = English()

Read in the CSV and name the columns. 
The third column is the one I am predicting on - summaries about the articles. 


In [38]:
newsPred = pd.read_csv('factCheck.csv')
#textRead = textRead.parse('Sheet1')
newsPred.columns = ['Json', 'Accuracy', 'Title', 'Genre', 'KeywordName', 'Occupation', 'Location', 'PoliticalParty', 
                    '1', '2', '3', '4', '5', 'Source', 'Url']


In [39]:
#separate the third column from the dataset 
articleTitle = newsPred[newsPred.columns[2]]
#sentenceFrame = pd.DataFrame(sentenceFrame)

Start exploring the data.  

Print the "-ing" verbs in the frame. We want to predict on the nouns, pronouns, etc. and other grammar types in this column. 

In [7]:
for line in articleTitle:
    for word in line.split():
        if word.endswith('ing'):
            print (word)

voting
starting
during
spending
making
cutting
Shooting
endorsing
going
going
existing
including
seeking
making
having
going
choosing
being
learning
being
driving
During
forthcoming
manufacturing
nothing
fastest-growing
kidnapping
peacekeeping
setting
privatizing
facing
leaving
during
totaling
Raising
engaging
opening
going
saying
growing
receiving
looking
creating
serving
spending
getting
creating
bring
amazing
going
bring
arriving
spending
begging
waiting


In [8]:
def stem(word):
    for suffix in ['ing', 'ly', 'ed', 'ious', 'ies', 'ive', 'es', 's', 'ment']:
        if word.endswith(suffix): return word[:-len(suffix)]
        return word


Lexical diversity is how common most of the words are, to see if some words appear more than others. 
In this case, since the diversity returns 1, most of the words are unique. Nothing special. 

In [9]:
def lexical_diversity(articleTitle):
    return (len(articleTitle)) / len(set(articleTitle))

lexical_diversity(articleTitle)

1.0

We take out the stopwords in the text and make everything lowercase for consistency. 

In [10]:
def preProcessing(features):
    with open("stop_words.txt", 'r') as f:
        stopwords = f.read().split("\n")
    
    num_titles = features.size
    clean_wordlist = []
    clean_titles = []
    for i in range( 0, num_titles):
        #letters_only = re.sub("[^a-zA-Z]", " ", features[i]) 
        words = features[i].lower().split()
        words = [w.lower() for w in words if not w in stopwords]  
        clean_wordlist.append(words)
        clean_titles.append(" ".join(words))
    return clean_titles, clean_wordlist

In [33]:
updatedTitles = preProcessing(articleTitle)
#print(updatedTitles)

#convert the data into a dataframe 
#everything is lowercase and the stopwords are taken out 
#updatedTitles = pd.dataframe(updatedTitles)

In [16]:
def find_features(updatedTitles):
    words = word_tokenize(updatedTitles)
    features = {}
    for w in words_features:
        features[w] = (w in words)
        print (features)
#function to return the tokenized frame but it does not work on my updated data for some reason 

We must tokenize the data frame to differentiate between the sentences and phrases. Stop words (common words such as "at", "by", etc.) have been removed. 

The function for preprocessing returns a tuple so I am going to create a dictionary to work with. 

In [10]:
#words_dict = {}
#for i in range(len(updatedFrame)):
 #   words_dict[updatedFrame[i]] = words[i]

In [26]:
#txt = updatedFrame.apply(word_tokenize)
#txt.head()

In [12]:
#with open("stop_words.txt", 'r') as f:
 #   stopwords = f.read().split("\n")

#filtered_sentence = [w for w in txt if not w in stopwords]

#filtered_sentence = []

#for w in txt:
 #   if w not in stopwords:
#filtered_sentence.append(w)

#print(txt)
#print(filtered_sentence)
#filtered_sentence = pd.DataFrame(filtered_sentence)
#filtered_sentence

In [26]:
texts = nltk.Text(articleTitle)



In [27]:
nltk.pos_tag(texts)
#NNP = proper noun, singular 



[('When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration.',
  'NNP'),
 ('Hillary Clinton agrees with John McCain "by voting to give George Bush the benefit of the doubt on Iran."',
  'NNP'),
 ('Health care reform legislation is likely to mandate free sex change surgeries.',
  'NNP'),
 ('The economic turnaround started at the end of my term.', 'NNP'),
 ('The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades.',
  'NNP'),
 ('Jim Dunnam has not lived in the district he represents for years now.',
  'NNP'),
 ("I'm the only person on this stage who has worked actively just last year passing, along with Russ Feingold, some of the toughest ethics reform since Watergate.",
  'NNP'),
 ('However, it took $19.5 million in Oregon Lottery funds for the Port of Newport to eventually land the new NOAA Marine Operatio

Here I am experimenting with different ways to create training and test sets. None are necessarily wrong, but I think the simplest way is to just subset. See line 166. 

In [53]:
for token in texts:
    if token.islower():
        print(token, 'is a lowercase word')
    elif token.istitle():
        print(token, 'is a titlecase word')
    else:
        print(token, 'is a punctuation')

When did the decline of coal start? It started when natural gas took off that started to begin in (President George W.) Bushs administration. is a punctuation
Hillary Clinton agrees with John McCain "by voting to give George Bush the benefit of the doubt on Iran." is a punctuation
Health care reform legislation is likely to mandate free sex change surgeries. is a punctuation
The economic turnaround started at the end of my term. is a punctuation
The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades. is a punctuation
Jim Dunnam has not lived in the district he represents for years now. is a punctuation
I'm the only person on this stage who has worked actively just last year passing, along with Russ Feingold, some of the toughest ethics reform since Watergate. is a punctuation
However, it took $19.5 million in Oregon Lottery funds for the Port of Newport to eventually land the new NOAA Ma

In [41]:
# Here we do a 80-20 split for our training and test set
train, test = train_test_split(newsPred, 
                               test_size = 0.20, 
                               random_state = 42)

# Create the training test omitting the diagnosis
training_set = train.ix[:, train.columns != 'Title']
# Next we create the class set (Called target in Python Documentation)
# Note: This was confusing af to figure out cus the documentation is low-key kind of shitty
class_set = train.ix[:, train.columns == 'Title']

# Next we create the test set doing the same process as the training set
test_set = test.ix[:, test.columns != 'Title']
test_class_set = test.ix[:, test.columns == 'Title']


Now I'm going to try to apply the algorithm. 

We are going to use the Naive Bayes classification algorithm. 
This is a pretty popular algorithm used in text classification, implementing supervised machine learning. We're showing the machine data, and telling it "hey, this data is positive," or "this data is negative." Then, after that training is done, we show the machine some new data and ask the computer, based on what we taught the computer before, what the computer thinks the category of the new data is.

I get stuck here. Why isn't my dataframe iterable? Any suggestions?

In [42]:
X = articleTitle
y = newsPred.ix[:, newsPred.columns != 'Title']
# we create an instance of SVM and fit out data. We do not scale our
# data since we want to plot the support vectors
C = 1.0 # SVM regularization parameter
#svc = svm.SVC(kernel='linear', C=1,gamma=0).fit(X, y)

In [49]:
from classifier import Classifier
from corpus import Document

from cStringIO import StringIO
from unittest import TestCase, main

class TrivialDocument(Document):
    """A document whose sole feature is its identity."""
    
    def features(self):
        return [id(self)]

class TrivialClassifier(Classifier):
    """A classifier that classifies based on exact feature matches."""

    def __init__(self, model={}):
        super(TrivialClassifier, self).__init__(model)

    def get_model(self): return self.seen
    def set_model(self, model): self.seen = model
    model = property(get_model, set_model)

    def train(self, instances):
        """Remember the labels associated with the features of instances."""
        for instance in instances:
            for feature in instance.features():
                self.seen[feature] = instance.label

    def classify(self, instance):
        """Classify an instance using the features seen during training."""
        for feature in instance.features():
            if feature in self.seen:
                return self.seen[feature]


class ClassifierTest(TestCase):
    def test_save_load(self):
        """Save and load a trivial model"""
        foo = TrivialDocument("foo", True)
        c1 = TrivialClassifier()
        c1.train([foo])
        model = c1.model
        stream = StringIO()
        c1.save(stream)
        self.assertGreater(len(stream.getvalue()), 0)

        c2 = TrivialClassifier()
        c2.load(StringIO(stream.getvalue()))
        self.assertEquals(c2.model, model)
        self.assertTrue(c2.classify(foo))

    def test_trivial_classifier(self):
        """Classify instances as seen or unseen"""
        corpus = [TrivialDocument(x, True) for x in ("foo", "bar", "baz")]
        classifier = TrivialClassifier()
        classifier.train(corpus)
        for x in corpus:
            self.assertTrue(classifier.classify(x))
        for y in [TrivialDocument(x) for x in ("foobar", "quux")]:
            self.assertFalse(classifier.classify(y))

if __name__ == '__main__':
    # Run all of the tests, print the results, and exit.
    main()


ImportError: No module named 'classifier'